# **Trabajo Práctico Final 2025 - Procesamiento del Lenguaje Natural - TUIA**

**Alumna**: Valentina Balverdi

**Profesores:**
* Juan Pablo Manson
* Alan Geray
* Constantino Ferrucci


### Modelo a usar

In [ ]:
modelo_gemini = "gemini-2.0-flash"

### Instalaciones

In [3]:
!pip install sentence-transformers chromadb
!pip install langchain-text-splitters
!pip install chromadb
!pip install txtai

### Descarga de archivos desde Drive

In [4]:
import os
import glob
import json
import numpy
import pandas as pd
import re

#### CSV a documento

In [5]:
from google.colab import drive
drive.mount('/content/drive')

BASE_PATH = "/content/drive/MyDrive/TUIA/NLP/TP_FINAL/fuentes_de_informacion"

# Carga de archivos tabulares
path = BASE_PATH

devoluciones = pd.read_csv(os.path.join(path, "devoluciones.csv"))
inventario = pd.read_csv(os.path.join(path, "inventario_sucursales.csv"))
productos = pd.read_csv(os.path.join(path, "productos.csv"))
ventas = pd.read_csv(os.path.join(path, "ventas_historicas.csv"))
vendedores = pd.read_csv(os.path.join(path, "vendedores.csv"))
tickets = pd.read_csv(os.path.join(path, "tickets_soporte.csv"))

with open(os.path.join(path, "faqs.json"), "r", encoding="utf-8") as f:
    faqs = json.load(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### TXT a documento

In [6]:
# Carga de textos
resenas_dir = os.path.join(path, "resenas_usuarios")
reviews_files = glob.glob(os.path.join(resenas_dir, "*.txt"))

reviews_docs = []

for fp in reviews_files:
    with open(fp, "r", encoding="utf-8") as f:
        text = f.read()

    # Separar encabezado de cuerpo (una línea en blanco entre ambos)
    partes = text.split("\n\n", 1)
    header = partes[0]
    cuerpo = partes[1] if len(partes) > 1 else ""

    # Inicializar campos
    fecha = usuario = telefono = producto_nombre = producto_id = puntaje = provincia = None

    for line in header.splitlines():
        line = line.strip()
        if line.startswith("Fecha:"):
            fecha = line.replace("Fecha:", "").strip()
        elif line.startswith("Usuario:"):
            usuario = line.replace("Usuario:", "").strip()
        elif line.startswith("Teléfono:"):
            telefono = line.replace("Teléfono:", "").strip()
        elif line.startswith("Producto:"):
            # Ej: "Producto: Profesional Batidora de Mano (P0017)"
            m = re.match(r"Producto:\s*(.+)\s+\((P\d+)\)", line)
            if m:
                producto_nombre = m.group(1).strip()
                producto_id = m.group(2).strip()
        elif line.startswith("Puntaje:"):
            # Ej: "Puntaje: 5/5"
            puntaje = line.replace("Puntaje:", "").strip()
        elif line.startswith("Provincia:"):
            provincia = line.replace("Provincia:", "").strip()

    reviews_docs.append({
        "content": cuerpo.strip(),              # texto que se embebe
        "source": os.path.basename(fp),
        "type": "review",
        "fecha": fecha,
        "usuario": usuario,
        "telefono": telefono,
        "producto_id": producto_id,
        "producto_nombre": producto_nombre,
        "puntaje": puntaje,
        "provincia": provincia,
    })


In [7]:
reviews_docs[0]

{'content': 'Buenas! Llegó rápido y funciona de maravilla con Aire Portátil Pro. Lo compré hace una semana y es muy fácil de usar, además de moderno. Ya se lo recomendé a varios amigos. Saludos!',
 'source': 'resena_R03514.txt',
 'type': 'review',
 'fecha': '2024-07-24',
 'usuario': 'Bianca_Romero',
 'telefono': '+54 9 367 8149-1834',
 'producto_id': 'P0156',
 'producto_nombre': 'Aire Portátil Pro',
 'puntaje': '5/5',
 'provincia': 'Neuquén'}

#### .md y JSON a documento

In [8]:

# Manuales .md
manuales_dir = os.path.join(path, "manuales_productos")
manuales_files = glob.glob(os.path.join(manuales_dir, "*.md"))

manuales_docs = []

for fp in manuales_files:
    with open(fp, "r", encoding="utf-8") as f:
        text = f.read()

    # --- Extraer información del encabezado ---
    # Título: "# Manual Técnico - Compacto Licuadora"
    titulo_match = re.search(r"# Manual Técnico - (.+)", text)
    producto_nombre = titulo_match.group(1).strip() if titulo_match else None

    # Modelo: "**Modelo:** P0004 | **Marca:** ChefMaster"
    modelo_match = re.search(r"\*\*Modelo:\*\*\s*([A-Z0-9]+)", text)
    producto_id = modelo_match.group(1).strip() if modelo_match else None

    marca_match = re.search(r"\*\*Marca:\*\*\s*([A-Za-z0-9]+)", text)
    marca = marca_match.group(1).strip() if marca_match else None

    # Nombre Comercial (en Especificaciones Técnicas)
    comercial_match = re.search(r"\*\*Nombre Comercial:\*\*\s*(.+)", text)
    nombre_comercial = comercial_match.group(1).strip() if comercial_match else producto_nombre

    # Categoría
    categoria_match = re.search(r"\*\*Categoría:\*\*\s*(.+)", text)
    categoria = categoria_match.group(1).strip() if categoria_match else None

    # Guardar documento con metadata
    manuales_docs.append({
        "content": text,
        "source": os.path.basename(fp),
        "type": "manual",
        "producto_id": producto_id,
        "producto_nombre": nombre_comercial,
        "marca": marca,
        "categoria_producto": categoria
    })

# Convertir FAQs a documentos
faq_docs = []

for item in faqs:
    pregunta = item["pregunta"]
    respuesta = item["respuesta"]
    producto = item["nombre_producto"]
    categoria = item["categoria"]

    faq_docs.append({
        "content": (
            f"PRODUCTO: {producto}\n"
            f"CATEGORÍA FAQ: {categoria}\n"
            f"PREGUNTA: {pregunta}\n"
            f"RESPUESTA: {respuesta}"
        ),
        "source": f"faq_{item['id_faq']}.json",
        "type": "faq",
        "id_faq": item["id_faq"],
        "id_producto": item["id_producto"],
        "producto_nombre": item["nombre_producto"],
        "categoria_faq": item["categoria"]
    })


# Tickets de soporte (texto largo)
ticket_docs = []

for _, row in tickets.iterrows():
    texto = ""

    # incluir tipo de problema si existe
    if isinstance(row["tipo_problema"], str):
        texto += f"Tipo de problema: {row['tipo_problema']}\n"

    # incluir descripción si existe
    if isinstance(row["descripcion"], str):
        texto += f"Descripción: {row['descripcion']}\n"

    # Si tenemos texto válido, lo guardamos
    if texto.strip():
        ticket_docs.append({
            "content": texto.strip(),
            "source": f"ticket_{row['id_ticket']}",
            "type": "ticket_soporte",
            "producto_id": row["id_producto"],
            "producto_nombre": row["nombre_producto"],
            "categoria": row["categoria"],
            "severidad": row["severidad"],
        })

# Union para la base vectorial
text_documents = reviews_docs + manuales_docs + faq_docs + ticket_docs


In [9]:
manuales_docs[0]

{'content': '# Manual Técnico - Procesadora\n\n**Modelo:** P0013 | **Marca:** KitchenPro\n\n---\n\n## Índice\n\n1. [Especificaciones Técnicas](#especificaciones-técnicas)\n2. [Componentes Principales](#componentes-principales)\n3. [Procedimientos de Uso](#procedimientos-de-uso)\n4. [Compatibilidad y Relaciones](#compatibilidad-y-relaciones)\n5. [Solución de Problemas](#solución-de-problemas)\n6. [Mantenimiento Preventivo](#mantenimiento-preventivo)\n7. [Información de Garantía](#información-de-garantía)\n8. [Contacto y Soporte](#contacto-y-soporte)\n\n---\n\n## Especificaciones Técnicas\n\n- **Modelo:** P0013\n- **Nombre Comercial:** Procesadora\n- **Categoría:** Cocina - Preparación\n- **Marca:** KitchenPro\n- **Color:** Azul\n- **Potencia:** 1700W\n- **Capacidad:** 2.0L\n- **Voltaje:** 220V\n- **Peso Neto:** 30.1 kg\n- **Garantía:** 24 meses\n- **Certificaciones:** CE, RoHS, ISO 9001\n- **Clase Energética:** A+\n- **Origen:** Importado\n\n## Componentes Principales\n\n### Motor Motor

In [10]:
faq_docs[0]

{'content': 'PRODUCTO: Licuadora\nCATEGORÍA FAQ: Especificaciones\nPREGUNTA: ¿Qué voltaje requiere?\nRESPUESTA: El Licuadora funciona con 12V. El consumo es de 650W. Recomendamos usar un estabilizador de tensión.',
 'source': 'faq_FAQ00001.json',
 'type': 'faq',
 'id_faq': 'FAQ00001',
 'id_producto': 'P0001',
 'producto_nombre': 'Licuadora',
 'categoria_faq': 'Especificaciones'}

In [11]:
ticket_docs[0]

{'content': 'Tipo de problema: Sobrecalentamiento\nDescripción: El producto se calienta excesivamente durante el uso',
 'source': 'ticket_TKT000001',
 'type': 'ticket_soporte',
 'producto_id': 'P0251',
 'producto_nombre': 'Pro Lavaseca',
 'categoria': 'Eléctrico',
 'severidad': 'Alta'}

Información de los datos

In [12]:
print("=== FAQs ===")
print("Cantidad de FAQs:", len(faqs))
print("Ejemplo FAQ[0]:")
print(faqs[0])

print("\n=== ticket_docs ===")
print("Cantidad de ticket_docs:", len(ticket_docs))
print("Ejemplo ticket_docs[0]:")
print(ticket_docs[0])

print("\n=== DataFrames tabulares ===")
for nombre, df in {
    "devoluciones": devoluciones,
    "inventario": inventario,
    "productos": productos,
    "ventas": ventas,
    "vendedores": vendedores,
    "tickets_df": tickets,
}.items():
    print(f"\n[{nombre}] shape={df.shape}")
    print("Columnas:", list(df.columns))


=== FAQs ===
Cantidad de FAQs: 3000
Ejemplo FAQ[0]:
{'id_faq': 'FAQ00001', 'id_producto': 'P0001', 'nombre_producto': 'Licuadora', 'categoria': 'Especificaciones', 'pregunta': '¿Qué voltaje requiere?', 'respuesta': 'El Licuadora funciona con 12V. El consumo es de 650W. Recomendamos usar un estabilizador de tensión.', 'fecha_publicacion': '2025-01-05', 'vistas': 4067, 'util': 22}

=== ticket_docs ===
Cantidad de ticket_docs: 2000
Ejemplo ticket_docs[0]:
{'content': 'Tipo de problema: Sobrecalentamiento\nDescripción: El producto se calienta excesivamente durante el uso', 'source': 'ticket_TKT000001', 'type': 'ticket_soporte', 'producto_id': 'P0251', 'producto_nombre': 'Pro Lavaseca', 'categoria': 'Eléctrico', 'severidad': 'Alta'}

=== DataFrames tabulares ===

[devoluciones] shape=(800, 14)
Columnas: ['id_devolucion', 'id_venta', 'fecha_devolucion', 'id_producto', 'nombre_producto', 'cliente_nombre', 'motivo', 'descripcion_cliente', 'estado', 'monto_venta', 'monto_reembolso', 'metodo_ree

In [13]:
def stats_longitudes(docs, nombre):
    longitudes = [len(d["content"]) for d in docs]
    if not longitudes:
        print(f"No hay documentos en {nombre}")
        return None

    serie = pd.Series(longitudes)
    print(f"=== {nombre} ===")
    print("Cantidad de documentos:", len(longitudes))
    print("Mínimo:", int(serie.min()))
    print("Máximo:", int(serie.max()))
    print("Promedio:", int(serie.mean()))
    print("Mediana:", int(serie.median()))
    print("Percentil 75:", int(serie.quantile(0.75)))
    print("Percentil 90:", int(serie.quantile(0.90)))
    print()
    return serie

len_reviews  = stats_longitudes(reviews_docs,  "Reseñas")
len_manuales = stats_longitudes(manuales_docs, "Manuales")
len_faqs     = stats_longitudes(faq_docs,      "FAQs")
len_tickets  = stats_longitudes(ticket_docs,   "Tickets soporte")


=== Reseñas ===
Cantidad de documentos: 5015
Mínimo: 132
Máximo: 282
Promedio: 200
Mediana: 203
Percentil 75: 223
Percentil 90: 237

=== Manuales ===
Cantidad de documentos: 50
Mínimo: 6450
Máximo: 8675
Promedio: 6623
Mediana: 6516
Percentil 75: 6539
Percentil 90: 6968

=== FAQs ===
Cantidad de documentos: 3000
Mínimo: 190
Máximo: 365
Promedio: 302
Mediana: 308
Percentil 75: 321
Percentil 90: 332

=== Tickets soporte ===
Cantidad de documentos: 2000
Mínimo: 74
Máximo: 109
Promedio: 91
Mediana: 92
Percentil 75: 102
Percentil 90: 109



# **Ejercicio 1:** RAG

## **Base de datos Vectorial**
La base de datos vectorial se utiliza para indexar todas las fuentes de información no estructurada o semiestructurada del proyecto, incluyendo:

- Manuales de producto (`.md`)
- Reseñas de usuarios (`.txt`)
- Preguntas frecuentes (`faqs.json`)
- Descripciones de tickets de soporte (`tickets_soporte.csv`)

Este tipo de contenido es ideal para un esquema (RAG), ya que los usuarios realizan consultas en lenguaje natural y se requiere una búsqueda semántica, no solo coincidencia literal de palabras clave.

### **Fragmentación del texto (Text Splitter)**

Para preparar los documentos antes de generar embeddings, se utilizó el **RecursiveCharacterTextSplitter** de LangChain. Este splitter fue elegido porque:

- Es robusto para textos largos como manuales.
- Intenta mantener unidades semánticas coherentes (párrafos, oraciones) antes de cortar por caracteres.
- Permite controlar el tamaño del fragmento (*chunk_size*) y el solapamiento (*chunk_overlap*).
- Evita cortes innecesarios en textos cortos como reseñas, FAQs y tickets.

A partir del análisis de longitudes reales de los documentos:

| Fuente      | P75 longitud |
|-------------|--------------|
| Reseñas     | 368 chars    |
| FAQs        | 317 chars    |
| Tickets     | 102 chars    |
| Manuales    | 6539 chars   |

Se seleccionó:

- **chunk_size = 512** → suficientemente grande para mantener intactas reseñas, FAQs y tickets,  
  pero suficientemente pequeño para dividir manuales extensos de forma manejable.
- **chunk_overlap = 64** (≈ 12%) → asegura continuidad semántica entre fragmentos contiguos.

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=512,
    chunk_overlap=64,
)

In [15]:
chunks_final = []

# ---- Reviews ----
for i, review_doc in enumerate(reviews_docs):
    chunks = splitter.split_text(review_doc["content"])
    for j, chunk in enumerate(chunks):
        chunks_final.append({
            "id": f"review_{i}_chunk_{j}",
            "content": chunk,
            "metadata": {
                "type": "review",
                "fuente": review_doc["source"],
                "chunk_index": j,
                "producto_id": review_doc.get("producto_id"),
                "producto_nombre": review_doc.get("producto_nombre"),
                "puntaje": review_doc.get("puntaje"),
                "provincia": review_doc.get("provincia"),
            }
        })

# ---- Manuales ----
for i, manual_doc in enumerate(manuales_docs):
    chunks = splitter.split_text(manual_doc["content"])
    for j, chunk in enumerate(chunks):
        chunks_final.append({
            "id": f"manual_{i}_chunk_{j}",
            "content": chunk,
            "metadata": {
                "type": "manual",
                "fuente": manual_doc["source"],
                "chunk_index": j,
                "producto_id": manual_doc.get("producto_id"),
                "producto_nombre": manual_doc.get("producto_nombre"),
                "marca": manual_doc.get("marca"),
                "categoria_producto": manual_doc.get("categoria_producto"),
            }
        })

# ---- FAQs ----
for i, faq_doc in enumerate(faq_docs):
    chunks = splitter.split_text(faq_doc["content"])
    for j, chunk in enumerate(chunks):
        chunks_final.append({
            "id": f"faq_{i}_chunk_{j}",
            "content": chunk,
            "metadata": {
                "type": "faq",
                "fuente": faq_doc["source"],
                "chunk_index": j,
                "id_faq": faq_doc.get("id_faq"),
                "id_producto": faq_doc.get("id_producto"),
                "producto_nombre": faq_doc.get("producto_nombre"),
                "categoria_faq": faq_doc.get("categoria_faq"),
            }
        })

# ---- Tickets ----
for i, ticket_doc in enumerate(ticket_docs):
    chunks = splitter.split_text(ticket_doc["content"])
    for j, chunk in enumerate(chunks):
        chunks_final.append({
            "id": f"ticket_{i}_chunk_{j}",
            "content": chunk,
            "metadata": {
                "type": "ticket",
                "fuente": ticket_doc["source"],
                "chunk_index": j,
                "producto_id": ticket_doc.get("producto_id"),
                "producto_nombre": ticket_doc.get("producto_nombre"),
                "categoria_ticket": ticket_doc.get("categoria"),
                "severidad": ticket_doc.get("severidad"),
            }
        })

print("Cantidad total de chunks:", len(chunks_final))
chunks_final[0]


Cantidad total de chunks: 10839


{'id': 'review_0_chunk_0',
 'content': 'Buenas! Llegó rápido y funciona de maravilla con Aire Portátil Pro. Lo compré hace una semana y es muy fácil de usar, además de moderno. Ya se lo recomendé a varios amigos. Saludos!',
 'metadata': {'type': 'review',
  'fuente': 'resena_R03514.txt',
  'chunk_index': 0,
  'producto_id': 'P0156',
  'producto_nombre': 'Aire Portátil Pro',
  'puntaje': '5/5',
  'provincia': 'Neuquén'}}

### **Modelo de Embeddings**

Para transformar cada fragmento en un vector numérico se seleccionó:

Modelo elegido: **sentence-transformers/paraphrase-multilingual-mpnet-base-v2**

Justificación:

- Es un modelo multilingüe optimizado para español.
- Destacado en tareas de similitud semántica y sistemas RAG.
- Produce embeddings densos de 768 dimensiones con alta calidad semántica.
- Eficiente
- Permite comparar consultas y documentos en el mismo espacio vectorial.

In [16]:
from sentence_transformers import SentenceTransformer

EMBEDDING_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### **Almacenamiento en ChromaDB**

Se utilizó ChromaDB con almacenamiento persistente, configurado con distancia coseno

In [17]:
import chromadb
from chromadb.config import Settings


In [18]:
chroma_client = chromadb.PersistentClient(path="chroma_vector_db")

# get_or_create obtiene la colección. Si ya existe en disco, la trae tal cual.
collection = chroma_client.get_or_create_collection(
    name="electrodomesticos_vector_store",
    metadata={"hnsw:space": "cosine"}
)

# VALIDACIÓN DE EXISTENCIA: Solo cargamos si está vacía
cantidad_actual = collection.count()

if cantidad_actual == 0:
    print("⚡ Colección vacía. Iniciando procesamiento e ingesta de documentos...")

    # Recién acá hacemos el embedding (lo que tarda)
    documents = [c["content"] for c in chunks_final]
    metadatas = [c["metadata"] for c in chunks_final]
    ids       = [c["id"]       for c in chunks_final]

    embeddings = embedding_model.encode(
        documents,
        batch_size=64,
        show_progress_bar=True
    )

    # Carga en lotes (Batching)
    BATCH_SIZE = 5000
    total_docs = len(documents)

    for i in range(0, total_docs, BATCH_SIZE):
        end = min(i + BATCH_SIZE, total_docs)
        print(f"Insertando batch {i} a {end}...")
        collection.add(
            documents=documents[i:end],
            metadatas=metadatas[i:end],
            ids=ids[i:end],
            embeddings=embeddings[i:end]
        )
    print("✅ Carga finalizada.")
else:
    print(f"✅ La base vectorial ya contiene {cantidad_actual} documentos. Se omite la ingesta.")

✅ La base vectorial ya contiene 10839 documentos. Se omite la ingesta.


**Función de Búsqueda Vectorial**

Esta función es el método que utiliza el sistema para recuperar fragmentos relevantes.
Incluye:
- Normalización de filtros ($and automático)
- Manejo de errores
- Ordenamiento por relevancia
- Soporte opcional para embeddings de los documentos (útil para reranking)

In [66]:
def normalizar_filtros(filtros: dict | None) -> dict | None:
    """
    Normaliza los filtros para Chroma:
    - Si no hay filtros (None o {}), devuelve None.
    - Si se pasa un dict simple {"campo": "valor"}, lo deja igual.
    - Si se arma un $and / $or, se asegura de que tenga al menos 2 condiciones.
      Si tiene solo 1, devuelve solo esa condición.
    """
    # Caso sin filtros
    if not filtros:  # None o {}
        return None

    # Si ya viene con operadores ($and, $or, $not) lo tratamos aparte
    if any(key.startswith("$") for key in filtros):
        # Caso $and / $or con lista vacía o de un solo elemento
        if "$and" in filtros:
            condiciones = filtros["$and"]
            if not condiciones:
                return None
            if len(condiciones) == 1:
                return condiciones[0]
            return filtros

        if "$or" in filtros:
            condiciones = filtros["$or"]
            if not condiciones:
                return None
            if len(condiciones) == 1:
                return condiciones[0]
            return filtros

        # Otros operadores ($not, etc.)
        return filtros

    # Si es un dict simple sin operadores
    condiciones = [{clave: valor} for clave, valor in filtros.items()]

    if len(condiciones) == 0:
        return None
    if len(condiciones) == 1:
        # Un solo filtro: lo devolvemos sin $and
        return condiciones[0]

    # Varios filtros simples: los combinamos con $and
    return {"$and": condiciones}



def buscar_vectorial(consulta, k=5, filtros=None, devolver_embeddings=False):
    """
    Realiza búsqueda vectorial en ChromaDB.

    Parámetros:
        consulta (str): texto de la consulta del usuario
        k (int): cantidad de resultados a recuperar
        filtros (dict): condiciones para filtrar metadata
        devolver_embeddings (bool): si se desea incluir embeddings del resultado
                                     (útil para rerankers tipo CrossEncoder)

    Devuelve:
        lista de dicts: cada uno con id, contenido, metadata y distancia
    """
    # 1. Embedding de la consulta
    query_emb = embedding_model.encode([consulta])[0]

    # 2. Normalizar filtros a formato aceptado por Chroma
    where = normalizar_filtros(filtros)

    # 3. Ejecutar consulta en Chroma
    if where:
        result = collection.query(
            query_embeddings=[query_emb],
            n_results=k,
            where=where
        )
    else:
        result = collection.query(
            query_embeddings=[query_emb],
            n_results=k
        )

    # 4. Si no encuentra nada
    if len(result["ids"][0]) == 0:
        return []

    # 5. Construir resultados limpios
    resultados = []
    for i in range(len(result["ids"][0])):
        entrada = {
            "id": result["ids"][0][i],
            "content": result["documents"][0][i],
            "metadata": result["metadatas"][0][i],
            "distance": result["distances"][0][i]
        }

        if devolver_embeddings:
            entrada["embedding_doc"] = result["embeddings"][0][i]

        resultados.append(entrada)

    # 6. Ordenar por distancia (menor = más relevante)
    resultados.sort(key=lambda x: x["distance"])

    return resultados


In [67]:
prueba = buscar_vectorial(
    "¿Cómo usar la licuadora para hacer un smoothie?",
    k=3,
    filtros={"type": "manual", "producto_nombre": "Compacto Licuadora"}
)

for r in prueba:
    print(r["metadata"])
    print(r["content"][:200], "...\n---")


{'marca': 'ChefMaster', 'fuente': 'manual_P0004_Compacto_Licuadora.md', 'chunk_index': 5, 'categoria_producto': 'Cocina - Preparación', 'producto_id': 'P0004', 'producto_nombre': 'Compacto Licuadora', 'type': 'manual'}
## Procedimientos de Uso

### PROCEDIMIENTO 1: Preparar Smoothie de Frutas

**Dificultad:** Fácil | **Tiempo:** 3-5 minutos

**Pasos:** ...
---
{'categoria_producto': 'Cocina - Preparación', 'producto_nombre': 'Compacto Licuadora', 'producto_id': 'P0004', 'fuente': 'manual_P0004_Compacto_Licuadora.md', 'chunk_index': 6, 'type': 'manual', 'marca': 'ChefMaster'}
1. Lavar y cortar las frutas en trozos medianos (2-3 cm)
2. Colocar los ingredientes líquidos primero (leche, yogurt, jugo)
3. Agregar las frutas y hielo en la jarra
4. Cerrar la tapa asegurándose del ...
---
{'categoria_producto': 'Cocina - Preparación', 'fuente': 'manual_P0004_Compacto_Licuadora.md', 'marca': 'ChefMaster', 'chunk_index': 7, 'type': 'manual', 'producto_id': 'P0004', 'producto_nombre': 'Compacto Li

In [68]:
def responder_desde_vectorial_con_llm(query_usuario: str, fragmentos: list[dict], client):
    """
    Toma la consulta del usuario + los fragmentos recuperados de Chroma
    y genera una respuesta en lenguaje natural usando el LLM.
    """
    if not fragmentos:
        contexto = "No se encontraron fragmentos relevantes."
    else:
        partes = []
        for i, frag in enumerate(fragmentos):
            meta = frag.get("metadata", {})
            fuente = meta.get("fuente", "desconocida")
            partes.append(
                f"[{i+1}] (fuente: {fuente})\n{frag['content']}"
            )
        contexto = "\n\n".join(partes)

    prompt = f"""
Sos un asistente de soporte de una empresa de electrodomésticos.

Usá EXCLUSIVAMENTE la siguiente información recuperada (fragmentos de manuales, reseñas, FAQs o tickets de soporte)
para responder la pregunta del usuario.

Si la información no alcanza para responder con seguridad, decí claramente que no hay suficiente información
y sugerí reformular la pregunta.

Pregunta del usuario:
{query_usuario}

Fragmentos recuperados:
{contexto}

Respuesta (en español, clara y directa):
"""

    response = client.models.generate_content(
        model=modelo_gemini,
        contents=[prompt]
    )

    return response.text.strip()


## **Base Tabular** (datos estadísticos)
Para resolver consultas donde el usuario necesita información estructurada.

Las tablas provienen de los archivos CSV del dataset:
- productos.csv
- ventas_historicas.csv
- inventario_sucursales.csv
- devoluciones.csv
- vendedores.csv
- tickets_soporte.csv

####  Agrupamos las tablas que vamos a usar como fuente tabular

In [22]:
tablas_tabulares = {
    "productos": productos,
    "inventario": inventario,
    "ventas": ventas,
    "devoluciones": devoluciones,
    "vendedores": vendedores,
    "tickets": tickets,
}

{nombre: df.shape for nombre, df in tablas_tabulares.items()}


{'productos': (300, 14),
 'inventario': (4100, 14),
 'ventas': (10000, 15),
 'devoluciones': (800, 14),
 'vendedores': (100, 10),
 'tickets': (2000, 17)}

### Generación de estructura de cada DataFrame para modelo

In [69]:
def generar_estructura_para_llm(nombre_df: str, df: pd.DataFrame, max_uniques: int = 20):
    """
    Genera una estructura compacta con información relevante de la tabla,
    pensada para ser convertida a string y enviada a un LLM.

    Incluye:
    - nombre de la tabla
    - columnas
    - tipos de dato
    - cantidad de filas / columnas
    - cantidad de nulos por columna
    - resumen por columna:
        - numéricas: min / max
        - categóricas: valores únicos (solo si son pocos)
        - texto: solo se marca como texto
    - estadísticas numéricas generales (describe), por si se quiere usar aparte
    """

    estructura = {
        "nombre_df": nombre_df,
        "cantidad_filas": int(len(df)),
        "cantidad_columnas": int(df.shape[1]),
        "columnas": df.columns.tolist(),
        "tipos_de_dato": df.dtypes.astype(str).to_dict(),
        "valores_nulos": df.isnull().sum().astype(int).to_dict(),
        "resumen_columnas": {},
        "estadisticas_numericas": {}
    }

    # Resumen por columna
    for col in df.columns:
        serie = df[col]
        col_info = {}

        if pd.api.types.is_numeric_dtype(serie):
            col_info["tipo_logico"] = "numerico"
            col_info["min"] = float(serie.min()) if not serie.isna().all() else None
            col_info["max"] = float(serie.max()) if not serie.isna().all() else None

        elif pd.api.types.is_object_dtype(serie) or pd.api.types.is_categorical_dtype(serie):
            # Columna categórica / texto
            n_unique = serie.nunique(dropna=True)
            col_info["tipo_logico"] = "categorico" if n_unique <= max_uniques else "texto_largo"
            col_info["num_valores_unicos"] = int(n_unique)

            if n_unique <= max_uniques:
                col_info["valores_unicos"] = serie.dropna().unique().tolist()

        else:
            col_info["tipo_logico"] = "otro"

        estructura["resumen_columnas"][col] = col_info

    # Estadísticas numéricas (describe)
    if not df.select_dtypes(include="number").empty:
        estructura["estadisticas_numericas"] = (
            df.select_dtypes(include="number").describe().to_dict()
        )

    return estructura


### Convertir la estructura a texto legible para el LLM

In [24]:
def estructura_a_string(estructura: dict) -> str:
    """
    Convierte la estructura generada por generar_estructura_para_llm
    a un string legible para usar en el prompt del LLM.
    """
    lineas = []
    lineas.append(f"Nombre de la tabla: {estructura['nombre_df']}")
    lineas.append(f"Filas: {estructura['cantidad_filas']}, Columnas: {estructura['cantidad_columnas']}")
    lineas.append("Columnas:")

    for col, info in estructura["resumen_columnas"].items():
        tipo = info.get("tipo_logico", "desconocido")

        if tipo == "numerico":
            lineas.append(
                f"- {col}: numérico, rango aproximado [{info.get('min')}, {info.get('max')}]"
            )
        elif tipo == "categorico":
            vals = ", ".join(map(str, info.get("valores_unicos", [])))
            lineas.append(
                f"- {col}: categórico, valores posibles: {vals}"
            )
        elif tipo == "texto_largo":
            lineas.append(
                f"- {col}: texto libre / muchas categorías (≈{info.get('num_valores_unicos')} valores distintos)"
            )
        else:
            lineas.append(f"- {col}: tipo {tipo}")

    return "\n".join(lineas)


### Esquemas que se le pasan al LLM

In [25]:
import json

# 1) Estructuras en diccionarios
esquemas_tabulares = {
    nombre: generar_estructura_para_llm(nombre, df)
    for nombre, df in tablas_tabulares.items()
}

# 2) Versión legible como texto para meter en el prompt del LLM
descripciones_tablas = {
    nombre: estructura_a_string(estructura)
    for nombre, estructura in esquemas_tabulares.items()
}



### Generación de prompt y conexión con gemini

In [ ]:
from google.colab import userdata
userdata.get('GOOGLE_API_KEY')

In [27]:
# Inicializamos el cliente Gemini
from google import genai
from google.colab import userdata

API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=API_KEY)

In [70]:
def armar_prompt_tabular(query_usuario: str) -> str:
    """
    Construye el prompt que se le va a enviar al LLM para que
    genere código de pandas que responda a la consulta del usuario.
    """

    partes = []

    partes.append(
        "Sos un asistente experto en análisis de datos con pandas.\n"
        "Trabajás con datos de una empresa de electrodomésticos.\n\n"
        "Tu tarea es: dado una pregunta del usuario, responder EXCLUSIVAMENTE con código Python\n"
        "que use DataFrames de pandas ya cargados para obtener la respuesta.\n\n"
        "IMPORTANTE:\n"
        "- NO crees DataFrames nuevos desde cero con datos escritos a mano.\n"
        "- NO uses read_csv ni lecturas de archivos.\n"
        "- NO imprimas nada, NO uses print.\n"
        "- NO expliques el código, solo devolvé la expresión de pandas.\n"
        "- La expresión debe devolver directamente el resultado (DataFrame o valor escalar).\n"
        "- Los DataFrames disponibles son:\n"
        "  * productos\n"
        "  * inventario\n"
        "  * ventas\n"
        "  * devoluciones\n"
        "  * vendedores\n"
        "  * tickets\n"
    )

    partes.append("\nA continuación te doy un resumen de las tablas disponibles:\n")

    for nombre, desc in descripciones_tablas.items():
        partes.append(f"\n### Tabla: {nombre}\n{desc}\n")

    partes.append("\nPregunta del usuario:\n")
    partes.append(query_usuario)
    partes.append(
        "\n\nAhora respondé ÚNICAMENTE con código Python válido que utilice pandas "
        "y estos DataFrames para obtener la respuesta. No agregues explicaciones ni comentarios."
    )

    return "\n".join(partes)


In [29]:
def ejecutar_consulta_tabular(query_usuario: str):
    """
    Usa el LLM (Gemini) para generar código de pandas a partir de la pregunta
    del usuario y luego ejecuta ese código sobre los DataFrames ya cargados.

    Devuelve:
        - resultado: lo que devuelve la expresión de pandas (DataFrame, serie, escalar, etc.)
        - codigo_generado: el string de código que generó el modelo
    """

    # 1) Armar el prompt a partir de tu función
    prompt = armar_prompt_tabular(query_usuario)

    # 2) Llamar al modelo
    response = client.models.generate_content(
        model=modelo_gemini,
        contents=[prompt]
    )

    # 3) Extraer el texto y limpiar bloque de código
    codigo_generado = response.text.strip()
    codigo_generado = (
        codigo_generado
        .replace("```python", "")
        .replace("```py", "")
        .replace("```", "")
        .strip()
    )

    print("🔧 Código generado por el modelo:\n")
    print(codigo_generado)
    print("\n---\n")

    # 4) Ejecutar el código. Necesita que los DataFrames estén en el scope global.
    try:
        resultado = eval(codigo_generado, globals())
    except Exception as e:
        print("⚠️ Error al ejecutar el código generado:")
        print(e)
        resultado = None

    return resultado, codigo_generado


In [30]:
def responder_desde_dataframe_con_llm(query_usuario, resultado, client):
    """
    Toma la pregunta del usuario + resultado pandas (DataFrame, Serie o escalar)
    y genera una respuesta clara y natural usando el LLM.
    """

    import pandas as pd

    if isinstance(resultado, pd.DataFrame):
        if resultado.empty:
            tabla = "Tabla vacía (sin resultados)."
        else:
            tabla = resultado.to_string(index=False)

    elif isinstance(resultado, pd.Series):
        tabla = resultado.to_string()

    else:
        tabla = str(resultado)

    prompt = f"""
Sos un asistente experto en análisis de datos tabulares.

Te doy:
- La pregunta original del usuario.
- El resultado ya filtrado desde las tablas de electrodomésticos.

Tu tarea:
- Responder en español, claro y directo.
- Explicar qué significa el resultado.
- NO mencionar pandas, ni código, ni cómo se obtuvo.
- Si está vacío, sugerir reformular la consulta.

Pregunta del usuario:
{query_usuario}

Resultado:
{tabla}

Respuesta:
"""

    response = client.models.generate_content(
        model=modelo_gemini,
        contents=[prompt]
    )

    return response.text.strip()

In [31]:
def pipeline_tabular(query_usuario: str):
    """
    Pipeline completo para la base TABULAR:
    - Genera código de pandas con el LLM
    - Ejecuta la expresión sobre los DataFrames
    - Genera una respuesta explicada en lenguaje natural
    """

    resultado, codigo = ejecutar_consulta_tabular(query_usuario)

    if resultado is None:
        return {
            "fuente": "tabular",
            "codigo_generado": codigo,
            "resultado_bruto": None,
            "respuesta": (
                "No pude ejecutar la consulta tabular. "
                "Probá reformular la pregunta o revisar los campos mencionados."
            ),
        }

    respuesta_nl = responder_desde_dataframe_con_llm(query_usuario, resultado, client)

    return {
        "fuente": "tabular",
        "codigo_generado": codigo,
        "resultado_bruto": resultado,
        "respuesta": respuesta_nl,
    }

In [32]:
resultado, codigo = ejecutar_consulta_tabular("Mostrame el top 10 de productos más vendidos por cantidad en la tabla de ventas.")
print("Resultado:\n", resultado)

🔧 Código generado por el modelo:

ventas.groupby('nombre_producto')['cantidad'].sum().nlargest(10)

---

Resultado:
 nombre_producto
Licuadora              408
Centro de Planchado    390
Waflera                365
Aire Split             300
Frigobar               265
Freidora de Aire       248
Neblinizador           245
Humidificador          234
Planchita de Pelo      231
Yogurtera              228
Name: cantidad, dtype: int64


## **Base de datos de grafos**

### Extracción de relaciones de compatibilidad desde los manuales

In [33]:
import re
import pandas as pd

compat_rows = []

for manual in manuales_docs:
    texto = manual["content"]
    producto_origen = manual.get("producto_id")
    nombre_origen = manual.get("producto_nombre")

    # Buscar sección "### Productos Compatibles ... (hasta la siguiente sección o el final)"
    patron_seccion = r"### Productos Compatibles(.*?)(## |$)"
    match = re.search(patron_seccion, texto, re.DOTALL)

    if not match:
        continue  # El manual no tiene sección de compatibilidad

    seccion = match.group(1)

    # Buscar líneas de productos compatibles:
    # - **Nombre Producto** (`P0001`)
    patron_producto = r"- \*\*(.*?)\*\* .*?\(`(P\d+)`\)"
    productos_encontrados = re.findall(patron_producto, seccion)

    # Buscar líneas con "Comparte: X"
    patron_comparte = r"Comparte:\s*([^\n\r]+)"
    comparte_list = re.findall(patron_comparte, seccion)

    # Emparejar producto destino con el texto de "comparte"
    for idx, (nombre_destino, id_destino) in enumerate(productos_encontrados):
        comparte = comparte_list[idx] if idx < len(comparte_list) else None

        compat_rows.append({
            "id_origen": producto_origen,
            "nombre_origen": nombre_origen,
            "id_destino": id_destino,
            "nombre_destino": nombre_destino,
            "comparte": comparte
        })

compat_df = pd.DataFrame(compat_rows)

print(compat_df.shape)
compat_df.head()

(250, 5)


,id_origen,nombre_origen,id_destino,nombre_destino,comparte
0,P0013,Procesadora,P0022,Advanced Batidora de Pie,Accesorios
1,P0013,Procesadora,P0035,Profesional Abridor de Latas,Jarra
2,P0013,Procesadora,P0131,Turbo Exprimidor,Accesorios
3,P0013,Procesadora,P0087,Olla de Cocción Lenta,Jarra
4,P0013,Procesadora,P0047,Turbo Microondas,Panel de control


### Construcción de la tabla de productos para el grafo

In [34]:
# Productos que participan en alguna relación de compatibilidad
ids_origen = compat_df["id_origen"]
ids_destino = compat_df["id_destino"]

ids_todos = pd.Series(
    pd.concat([ids_origen, ids_destino]).unique(),
    name="id_producto"
)

# Nos quedamos con columnas relevantes de la tabla productos
productos_grafo_df = (
    ids_todos.to_frame()
    .merge(
        productos[["id_producto", "nombre", "categoria", "marca"]],
        on="id_producto",
        how="left"
    )
)

print("Cantidad de productos en el grafo:", len(productos_grafo_df))
productos_grafo_df.head()

Cantidad de productos en el grafo: 192


,id_producto,nombre,categoria,marca
0,P0013,Procesadora,Cocina,KitchenPro
1,P0016,Super Picadora,Cocina,CookElite
2,P0082,Olla Arrocera 3000,Cocina,HomeChef
3,P0149,Aire Split,Climatización,EcoClima
4,P0050,Freidora de Aire,Cocina,KitchenPro


### Conexión a Neo4j y creación del grafo de productos

In [35]:
!pip install py2neo


In [36]:
from google.colab import userdata
from py2neo import Graph, Node, Relationship

NEO4J_URI = "neo4j+s://1830c5fb.databases.neo4j.io"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = userdata.get("NEO4_KEY")

graph = Graph(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Test rápido de conexión
graph.run("RETURN 1 AS test").data()

[{'test': 1}]

### Carga de nodos :Producto en Neo4j

In [37]:
def cargar_productos_py2neo(graph, df):
    tx = graph.begin()

    for _, row in df.iterrows():
        nodo = Node(
            "Producto",
            id=row["id_producto"],
            nombre=row["nombre"],
            categoria=row["categoria"],
            marca=row["marca"]
        )
        tx.merge(nodo, "Producto", "id")  # upsert por id

    tx.commit()
    print(f"Se cargaron/actualizaron {len(df)} nodos :Producto.")

cargar_productos_py2neo(graph, productos_grafo_df)

Se cargaron/actualizaron 192 nodos :Producto.


/tmp/ipython-input-1693625308.py:14: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


### Carga de relaciones :COMPATIBLE_CON en Neo4j

In [38]:
def cargar_relaciones_py2neo(graph, df):
    tx = graph.begin()

    for _, row in df.iterrows():
        origen = graph.nodes.match("Producto", id=row["id_origen"]).first()
        destino = graph.nodes.match("Producto", id=row["id_destino"]).first()

        if origen and destino:
            rel = Relationship(origen, "COMPATIBLE_CON", destino)
            rel["comparte"] = row["comparte"]
            tx.merge(rel)

    tx.commit()
    print(f"Se cargaron/actualizaron {len(df)} relaciones :COMPATIBLE_CON.")

cargar_relaciones_py2neo(graph, compat_df)

Se cargaron/actualizaron 250 relaciones :COMPATIBLE_CON.


/tmp/ipython-input-4095784289.py:13: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


#### Helper para ejecutar consultas Cypher desde Python

In [39]:
def ejecutar_cypher(query: str, params: dict | None = None):
    params = params or {}
    return graph.run(query, params).data()

### Prompt para que el LLM genere queries Cypher sobre el grafo

In [71]:
def armar_prompt_grafo(query_usuario: str) -> str:
    """
    Construye el prompt para que el LLM genere una query Cypher
    sobre el grafo de productos y compatibilidades.
    """

    return f"""
Sos un asistente experto en Neo4j y lenguaje Cypher.

TRABAJÁS CON ESTE GRAFO:

Nodos:
  (:Producto)
    - id        (por ejemplo: "P0013")
    - nombre    (por ejemplo: "Procesadora")
    - categoria
    - marca

Relaciones:
  (:Producto)-[:COMPATIBLE_CON {{comparte: <string>}}]->(:Producto)

REGLAS ESTRICTAS PARA GENERAR CYPHER:

- SIEMPRE usá un alias para la relación:
      (p:Producto)-[r:COMPATIBLE_CON]->(c:Producto)

- NUNCA pongas propiedades de la relación dentro del MATCH.
   NO hacer cosas como:
      -[:COMPATIBLE_CON {{comparte: algo}}]->

- Las propiedades de la relación se leen SOLO en el RETURN:
      r.comparte AS componente_compartido

- La query debe SIEMPRE devolver EXACTAMENTE estas columnas, en este orden:
      c.id        AS id_producto,
      c.nombre    AS nombre,
      c.categoria AS categoria,
      c.marca     AS marca,
      r.comparte  AS componente_compartido

- NO uses "null AS componente_compartido".
- NO agregues texto adicional, explicaciones ni comentarios.
- NO uses bloques ``` ni markdown.
- La consulta debe estar compuesta por UN SOLO statement Cypher.

El usuario puede preguntar en lenguaje natural por compatibilidades
entre productos (por nombre o por id). Usá esa información para
construir el WHERE o el patrón de MATCH correspondiente.

Pregunta del usuario:
{query_usuario}

Devolvé ÚNICAMENTE la query Cypher válida que cumple todas las reglas anteriores.
"""


### Conversión de lenguaje natural a Cypher y ejecución en Neo4j

In [72]:
def consulta_grafo_con_llm(query_usuario: str):
    """
    Usa el LLM para convertir una pregunta en lenguaje natural
    a una query Cypher, ejecuta esa query en Neo4j
    y devuelve (resultados, cypher_generado).
    """
    prompt = armar_prompt_grafo(query_usuario)

    response = client.models.generate_content(
        model=modelo_gemini,
        contents=[prompt]
    )

    cypher_generado = (
        response.text.replace("```cypher", "")
        .replace("```", "")
        .strip()
    )

    print("🔧 Cypher generado por el modelo:\n")
    print(cypher_generado)
    print("\n---\n")

    try:
        resultados = ejecutar_cypher(cypher_generado)
    except Exception as e:
        print("⚠️ Error al ejecutar la query Cypher:")
        print(e)
        resultados = []

    return resultados, cypher_generado


### Generación de respuesta en lenguaje natural desde el resultado del grafo

In [42]:
def responder_desde_grafo_con_llm(query_usuario: str, resultados: list[dict], client):
    """
    Toma la pregunta original + los resultados (lista de diccionarios)
    y le pide al LLM que genere una respuesta explicada.
    """
    if not resultados:
        tabla = "Sin resultados (la consulta al grafo no devolvió filas)."
    else:
        df_res = pd.DataFrame(resultados)
        tabla = df_res.to_string(index=False)

    prompt = f"""
Sos un asistente que trabaja con un grafo de productos de electrodomésticos.

Te doy la pregunta original del usuario y el resultado de una consulta a Neo4j.
El resultado ya contiene los productos compatibles (si los hay).

Tu tarea:
- Responder en español, de forma clara y directa.
- Explicar qué productos son compatibles, si comparten componentes, etc.
- Si no hay resultados, explicá que no se encontró compatibilidad para ese caso.
- NO expliques la query Cypher ni los detalles internos del grafo.

Pregunta del usuario:
{query_usuario}

Resultado de la consulta al grafo:
{tabla}

Respuesta:
"""

    response = client.models.generate_content(
        model=modelo_gemini,
        contents=[prompt]
    )

    return response.text.strip()

### Pipeline completo de la fuente "grafo"

In [43]:
def pipeline_grafo(query_usuario: str):
    resultados, cypher = consulta_grafo_con_llm(query_usuario)
    respuesta_nl = responder_desde_grafo_con_llm(query_usuario, resultados, client)
    return {
        "fuente": "grafo",
        "cypher_generado": cypher,
        "resultado_bruto": resultados,
        "respuesta": respuesta_nl,
    }

#Ejemplo de prueba puntual:
# resp_g = pipeline_grafo("¿Qué productos son compatibles con la Procesadora?")
# print(resp_g["respuesta"])

## **Clasificador de intención avanzado**

En esta sección se desarrolla un **clasificador de intención** que decide a qué fuente de datos
debe dirigirse el sistema según la pregunta del usuario:

- **Vectorial**: consultas típicas de manuales, FAQs, reseñas y problemas de uso de productos.
- **Tabular**: consultas sobre datos estructurados (ventas, stock, devoluciones, totales, rankings, etc.).
- **Grafo**: consultas sobre **compatibilidad** entre productos y repuestos.

Para cumplir con el enunciado se implementan dos clasificadores:

1. Un **clasificador entrenado propio**, basado en TF-IDF + Logistic Regression, utilizando un
   conjunto de preguntas sintéticas representativas de cada tipo de fuente.
2. Un **clasificador basado en LLM** (Gemini) con *few-shot prompting*, donde se le explican
   las clases al modelo y se le dan ejemplos de preguntas ya etiquetadas.

Finalmente, se comparan ambos clasificadores utilizando métricas de clasificación y se justifica
qué enfoque resulta más adecuado para el sistema.

### Dataset de ejemplos sintéticos

In [44]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

import google.generativeai as genai
genai.configure(api_key=API_KEY)

MODELO_GEMINI_CLF = modelo_gemini

In [45]:
# Cada tupla es: (texto_pregunta, etiqueta_real)
# etiquetas: "vectorial", "tabular", "grafo"

datos_clasificacion = [
    # VECTORIAL: manuales, FAQs, problemas de uso, reseñas
    ("¿Qué voltaje requiere la licuadora modelo P0001?", "vectorial"),
    ("¿Cómo se limpia el filtro del lavarropas?", "vectorial"),
    ("¿Este microondas tiene función grill?", "vectorial"),
    ("Mi heladera hace un ruido raro, ¿es normal?", "vectorial"),
    ("Mostrame las instrucciones para usar la procesadora", "vectorial"),
    ("¿Cuánto mide el cable de este ventilador?", "vectorial"),
    ("¿Cómo reinicio el horno si se queda trabado?", "vectorial"),
    ("¿Qué garantía tiene el modelo P0003?", "vectorial"),
    ("¿Dónde están las instrucciones de seguridad de la licuadora?", "vectorial"),
    ("¿Cómo cambio el filtro de la aspiradora?", "vectorial"),

    # TABULAR: ventas, stock, precios, KPIs
    ("Mostrame las ventas totales del mes pasado", "tabular"),
    ("¿Cuántas unidades de la heladera P0005 hay en stock?", "tabular"),
    ("Listá los vendedores con más devoluciones", "tabular"),
    ("¿Qué producto tuvo más tickets de soporte en 2024?", "tabular"),
    ("Dame el top 5 de productos más vendidos", "tabular"),
    ("Mostrame el total facturado por categoría este año", "tabular"),
    ("¿Cuál fue la sucursal con más ventas este trimestre?", "tabular"),
    ("Mostrame el stock actual de todos los lavarropas", "tabular"),
    ("¿Qué vendedor tuvo más ventas de licuadoras?", "tabular"),
    ("¿Cuántas devoluciones hubo de la procesadora P0010?", "tabular"),

    # GRAFO: compatibilidad entre productos/repuestos
    ("¿Qué repuestos son compatibles con la licuadora P0001?", "grafo"),
    ("¿Este filtro es compatible con qué modelos de lavarropas?", "grafo"),
    ("Listá todos los productos compatibles con el horno P0100", "grafo"),
    ("¿Qué modelos comparten el mismo motor que la aspiradora P0200?", "grafo"),
    ("¿Con qué otros productos es compatible la pieza X123?", "grafo"),
    ("Mostrame todos los productos que comparten repuestos con la licuadora P0003", "grafo"),
    ("¿Qué modelos usan el mismo filtro que la heladera P0300?", "grafo"),
    ("¿La resistencia R45 es compatible con qué hornos?", "grafo"),
    ("¿Qué otros modelos aceptan este mismo repuesto?", "grafo"),
    ("Mostrame todos los productos relacionados por compatibilidad con P0500", "grafo"),
]

df_clf = pd.DataFrame(datos_clasificacion, columns=["texto", "label"])

### Clasificador entrenado propio (TF-IDF + Logistic Regression)

In [46]:
# 1) Partición train / test
X = df_clf["texto"]
y = df_clf["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

# 2) Vectorizador TF-IDF
vectorizer_intencion = TfidfVectorizer(
    ngram_range=(1, 2),   # unigrama + bigrama
    min_df=1
)

X_train_vec = vectorizer_intencion.fit_transform(X_train)
X_test_vec  = vectorizer_intencion.transform(X_test)

# 3) Modelo de clasificación
clf_intencion = LogisticRegression(
    max_iter=200,
    multi_class="auto"
)
clf_intencion.fit(X_train_vec, y_train)

# 4) Evaluación en test (modelo entrenado propio)
y_pred = clf_intencion.predict(X_test_vec)

print("=== CLASIFICADOR ENTRENADO (TF-IDF + LogisticRegression) ===")
print(classification_report(y_test, y_pred))
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))


def clasificar_intencion_ml(pregunta: str) -> str:
    """
    Clasificador de intención basado en modelo entrenado (TF-IDF + LogisticRegression).

    Devuelve una de las etiquetas:
      - "vectorial"
      - "tabular"
      - "grafo"
    """
    X_vec = vectorizer_intencion.transform([pregunta])
    pred = clf_intencion.predict(X_vec)[0]
    return pred

=== CLASIFICADOR ENTRENADO (TF-IDF + LogisticRegression) ===
              precision    recall  f1-score   support

       grafo       0.75      1.00      0.86         3
     tabular       1.00      0.33      0.50         3
   vectorial       0.75      1.00      0.86         3

    accuracy                           0.78         9
   macro avg       0.83      0.78      0.74         9
weighted avg       0.83      0.78      0.74         9

Matriz de confusión:
[[3 0 0]
 [1 1 1]
 [0 0 3]]


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Para el clasificador entrenado se construyó un conjunto de **30 preguntas sintéticas**,
distribuidas equitativamente en las tres clases de intención:

- `vectorial`
- `tabular`
- `grafo`

Cada pregunta representa un tipo de consulta que luego podría hacer un usuario real del sistema
(por ejemplo, preguntas sobre uso de productos, sobre ventas/stock o sobre compatibilidad de repuestos).

El pipeline utilizado fue:

1. Vectorización de las preguntas con **TF-IDF**, considerando unigramas y bigramas.
2. División del dataset en entrenamiento y prueba mediante `train_test_split`, manteniendo el balance entre clases.
3. Entrenamiento de un modelo de **Logistic Regression**.
4. Evaluación sobre el conjunto de prueba usando `classification_report` y `confusion_matrix`.

En el conjunto de prueba se obtuvieron métricas cercanas a:

- **Accuracy** ≈ 0.78  

Este clasificador tiene como ventaja principal su **bajo costo de cómputo** y su independencia de servicios externos, por lo que es una buena opción cuando se prioriza eficiencia y simplicidad.

### Clasificador basado en LLM (Few-Shot) y comparación

In [47]:
def armar_prompt_clasificador_llm(pregunta: str) -> str:
    """
    Construye el prompt para que Gemini clasifique la intención
    en una de las tres clases: vectorial / tabular / grafo.
    """

    return f"""
Sos un asistente que clasifica preguntas de usuarios de una empresa de electrodomésticos
en TRES categorías según la FUENTE DE DATOS que habría que usar.

Las clases posibles son:

1) vectorial:
   - Preguntas sobre cómo usar un producto.
   - Consultas típicas de manuales, FAQs, problemas de uso, reseñas.
   - Ejemplos: instrucciones, voltaje, limpieza, garantía, problemas de funcionamiento.

2) tabular:
   - Preguntas sobre datos numéricos o estadísticos.
   - Ventas, stock, devoluciones, rankings, totales, top N, facturación, por sucursal.
   - Ejemplos: "top 10 más vendidos", "total facturado", "stock disponible", "vendedor con más ventas".

3) grafo:
   - Preguntas sobre compatibilidad entre productos o repuestos.
   - Qué modelos comparten componentes, qué repuestos sirven para qué modelos.
   - Ejemplos: "¿Con qué modelos es compatible?", "¿qué productos comparten el mismo motor?".

EJEMPLOS (few-shot):

PREGUNTA: "¿Qué voltaje requiere la licuadora modelo P0001?"
CLASE: vectorial

PREGUNTA: "¿Cómo se limpia el filtro del lavarropas?"
CLASE: vectorial

PREGUNTA: "Mostrame las ventas totales del mes pasado"
CLASE: tabular

PREGUNTA: "¿Cuántas unidades de la heladera P0005 hay en stock?"
CLASE: tabular

PREGUNTA: "¿Qué repuestos son compatibles con la licuadora P0001?"
CLASE: grafo

PREGUNTA: "Mostrame todos los productos que comparten repuestos con la licuadora P0003"
CLASE: grafo


Ahora clasificá la siguiente pregunta del usuario EN UNA SOLA PALABRA,
usando exactamente una de estas opciones:

- vectorial
- tabular
- grafo

No agregues explicaciones ni texto extra.

PREGUNTA A CLASIFICAR:
{pregunta}

RESPUESTA:
""".strip()


def clasificar_intencion_llm(pregunta: str) -> str:
    """
    Clasificador de intención basado en LLM (Gemini) con few-shot prompting.

    Devuelve:
      - "vectorial"
      - "tabular"
      - "grafo"
    """

    prompt = armar_prompt_clasificador_llm(pregunta)

    response = client.models.generate_content(
        model=modelo_gemini,
        contents=[prompt]
    )

    texto = response.text.strip().lower()

    # Normalizamos por si el modelo devuelve algo tipo "La clase es: tabular"
    if "vectorial" in texto:
        return "vectorial"
    if "tabular" in texto:
        return "tabular"
    if "grafo" in texto:
        return "grafo"

    # Fallback: si no podemos parsear, usamos el modelo entrenado
    return clasificar_intencion_ml(pregunta)


### Comparación de clasificadores

In [48]:
def comparar_clasificadores(df=df_clf, max_muestras_llm: int = 20):
    """
    Compara el clasificador entrenado (ML) y el clasificador LLM few-shot.

    - Para el modelo entrenado se calcula classification_report completo.
    - Para el LLM se evalúa sobre un subconjunto (max_muestras_llm) para
      evitar demasiadas llamadas a la API.

    Imprime las métricas por pantalla.
    """

    # --- Modelo entrenado (ML) ya lo evaluamos arriba con X_test e y_test ---
    print("\n\n=== RESUMEN: CLASIFICADOR ENTRENADO (ML) ===")
    X_vec = vectorizer_intencion.transform(df["texto"])
    y_true = df["label"]
    y_pred_ml = clf_intencion.predict(X_vec)
    print(classification_report(y_true, y_pred_ml))

    # --- Clasificador LLM (few-shot) en un subconjunto ---
    print("\n=== RESUMEN: CLASIFICADOR LLM (FEW-SHOT) ===")
    # Subconjunto para no gastar demasiadas llamadas
    sub = df.sample(min(max_muestras_llm, len(df)), random_state=42)

    y_true_llm = []
    y_pred_llm = []

    for _, row in sub.iterrows():
        txt = row["texto"]
        etiqueta_real = row["label"]
        pred_llm = clasificar_intencion_llm(txt)

        y_true_llm.append(etiqueta_real)
        y_pred_llm.append(pred_llm)

        print(f"Pregunta: {txt}")
        print(f"  Real: {etiqueta_real} | LLM: {pred_llm}")
        print("---")

    print("\n=== MÉTRICAS LLM EN SUBCONJUNTO ===")
    print(classification_report(y_true_llm, y_pred_llm))

#### Funcion unificada para el asistente

In [49]:
def clasificar_intencion(query_usuario: str, metodo: str = "ml") -> str:
    """
    Envuelve ambos clasificadores y te deja elegir el método:

    metodo = "ml"   -> usa clasificador entrenado (TF-IDF + LogisticRegression)
    metodo = "llm"  -> usa clasificador basado en LLM few-shot
    metodo = "mixto"-> usa ML y, si quisiera, podría combinar (acá por ahora = ML)
    """
    metodo = metodo.lower()

    if metodo == "llm":
        return clasificar_intencion_llm(query_usuario)
    return clasificar_intencion_ml(query_usuario)


In [50]:
def asistente_electro(
    query_usuario: str,
    historial=None,
    k_vectorial: int = 5,
    metodo_clasificador: str = "ml"
):
    """
    Punto de entrada único del asistente.
    - Usa el clasificador de intención (ML o LLM)
    - Llama al pipeline correspondiente (vectorial, tabular o grafo)
    """

    fuente = clasificar_intencion(query_usuario, metodo=metodo_clasificador)

    if fuente == "vectorial":

        return pipeline_vectorial(query_usuario, k_final=k_vectorial)

        # # Si preferís algo más simple por ahora:
        # fragmentos = buscar_vectorial(query_usuario, k=k_vectorial)
        # respuesta = responder_desde_vectorial_con_llm(query_usuario, fragmentos, client)
        # return {
        #     "fuente": "vectorial",
        #     "fragmentos": fragmentos,
        #     "respuesta": respuesta,
        # }

    elif fuente == "tabular":
        return pipeline_tabular(query_usuario)

    elif fuente == "grafo":
        return pipeline_grafo(query_usuario)

    # Fallback raro, por las dudas
    return {
        "fuente": "desconocida",
        "respuesta": (
            "Por ahora no sé a qué fuente de datos dirigir esta consulta. "
            "Probá reformular la pregunta con más detalle sobre si buscás precios, "
            "compatibilidades o instrucciones de uso."
        )
    }


In [51]:
# Ejemplo de uso del asistente integrado

resp = asistente_electro("¿Qué productos son compatibles con la Procesadora?")
print(resp["fuente"])
print()
print(resp["respuesta"])

🔧 Cypher generado por el modelo:

MATCH (p:Producto)-[r:COMPATIBLE_CON]->(c:Producto) WHERE p.nombre = "Procesadora" RETURN c.id AS id_producto, c.nombre AS nombre, c.categoria AS categoria, c.marca AS marca, r.comparte AS componente_compartido

---

grafo

Los siguientes productos son compatibles con la Procesadora, ya que comparten componentes:

*   **Advanced Batidora de Pie:** Comparte el componente "Accesorios".
*   **Profesional Abridor de Latas:** Comparte el componente "Jarra".
*   **Turbo Exprimidor:** Comparte el componente "Accesorios".
*   **Olla de Cocción Lenta:** Comparte el componente "Jarra".
*   **Turbo Microondas:** Comparte el componente "Panel de control".


In [52]:
comparar_clasificadores()



=== RESUMEN: CLASIFICADOR ENTRENADO (ML) ===
              precision    recall  f1-score   support

       grafo       0.91      1.00      0.95        10
     tabular       1.00      0.80      0.89        10
   vectorial       0.91      1.00      0.95        10

    accuracy                           0.93        30
   macro avg       0.94      0.93      0.93        30
weighted avg       0.94      0.93      0.93        30


=== RESUMEN: CLASIFICADOR LLM (FEW-SHOT) ===
Pregunta: ¿La resistencia R45 es compatible con qué hornos?
  Real: grafo | LLM: grafo
---
Pregunta: Mostrame el total facturado por categoría este año
  Real: tabular | LLM: tabular
---
Pregunta: ¿Qué modelos comparten el mismo motor que la aspiradora P0200?
  Real: grafo | LLM: grafo
---
Pregunta: Mostrame el stock actual de todos los lavarropas
  Real: tabular | LLM: tabular
---
Pregunta: ¿Dónde están las instrucciones de seguridad de la licuadora?
  Real: vectorial | LLM: vectorial
---
Pregunta: ¿Cómo cambio el filtr

Además del modelo entrenado tradicional, se implementó un **clasificador de intención basado en LLM**
utilizando Gemini. Para esto se diseñó un prompt que:

- Describe en lenguaje natural las tres clases posibles (`vectorial`, `tabular`, `grafo`).
- Incluye **ejemplos etiquetados** de preguntas típicas de cada clase (enfoque *few-shot*).
- Le indica explícitamente al modelo que debe responder **solo con el nombre de la clase**.

A partir de este prompt, la función `clasificar_intencion_llm` toma una pregunta en lenguaje natural,
consulta al modelo y normaliza su salida para obtener una de las tres etiquetas.

Para comparar ambos enfoques se utilizó la función `comparar_clasificadores`, que evalúa:

- El **clasificador entrenado (ML)** sobre todo el dataset sintético.
- El **clasificador LLM** sobre un subconjunto de ejemplos, para evitar un número excesivo de llamadas a la API.

En los experimentos realizados se observaron resultados del estilo:

- Clasificador entrenado (ML):  
  - accuracy ≈ 0.93  
  - buen F1-score en las tres clases

- Clasificador LLM (few-shot):  
  - accuracy ≈ 1.00 en el subconjunto evaluado  
  - clasificación correcta en todos los ejemplos probados

Estos resultados muestran que el LLM, guiado con ejemplos adecuados, es capaz de captar muy bien
la diferencia semántica entre las tres intenciones. Sin embargo, su uso implica:

- **Mayor latencia** (cada predicción requiere una llamada a la API).
- **Costo asociado** al uso del modelo en la nube.
- Dependencia de la disponibilidad del servicio externo.

Por otro lado, el clasificador entrenado con TF-IDF + Logistic Regression ofrece:

- Muy buen rendimiento (más del 90% de accuracy).
- Costo de cómputo despreciable una vez entrenado.
- Independencia de APIs externas.

Por este motivo, en la implementación final del asistente se ofrece la posibilidad de usar
ambos clasificadores a través del parámetro `metodo_clasificador`, pero se deja como opción
por defecto el **clasificador entrenado (ML)**, priorizando eficiencia y simplicidad en el
entorno de producción, y manteniendo el clasificador LLM como alternativa más “inteligente”
pero también más costosa.

## Recuperación Híbrida (BM25 + Embeddings)

Para mejorar la calidad de la recuperación en la fuente vectorial, se implementa un
pipeline híbrido que combina dos enfoques complementarios:

1. **Búsqueda por palabras clave (BM25)**  
   Permite detectar coincidencias literales y términos específicos, siendo muy útil cuando
   el usuario menciona palabras exactas del manual o la FAQ.

2. **Búsqueda semántica mediante embeddings**  
   Utiliza el espacio vectorial generado previamente, detectando similitud semántica
   incluso cuando no se repiten las mismas palabras.

La combinación de ambos métodos permite cubrir casos donde:

- El usuario recuerda parte de una frase del manual (BM25 funciona bien).
- El usuario parafrasea o hace una pregunta conceptual no textual (embeddings funcionan mejor).

In [53]:
# Corpus de texto para BM25 (uno por chunk)
bm25_corpus = [c["content"] for c in chunks_final]

# Mapeo índice entero -> chunk original
bm25_id_to_chunk = chunks_final  # lista, índice = entero usado por BM25


### Búsqueda por palabras clave con BM25

Se construyó un índice BM25 utilizando la librería `txtai`, donde cada documento
representa un *chunk* de los manuales, reseñas y FAQs.

El objetivo de esta etapa es recuperar rápidamente candidatos que contienen términos
mencionados por el usuario. La función `buscar_bm25()` recibe una consulta y devuelve
los `k` chunks más relevantes según BM25.

BM25 es especialmente útil cuando el usuario utiliza términos específicos del manual o
palabras que aparecen textualmente en los documentos.

In [73]:
from txtai.scoring import ScoringFactory

# Índice BM25
bm25_scoring = ScoringFactory.create({
    "method": "bm25",
    "terms": True  # índice de palabras clave
})

# Indexar todos los chunks
bm25_scoring.index(
    ( (idx, text, None) for idx, text in enumerate(bm25_corpus) )
)

print("Índice BM25 construido. Cantidad de documentos:", bm25_scoring.count())


Índice BM25 construido. Cantidad de documentos: 10839


In [55]:
def buscar_bm25(consulta: str, k: int = 10):
    """
    Búsqueda por palabras clave usando BM25 sobre los chunks.
    Devuelve una lista de dicts con:
    - id, content, metadata, score_bm25
    """
    resultados_raw = bm25_scoring.search(consulta, k)

    resultados = []
    for idx, score in resultados_raw:
        chunk = bm25_id_to_chunk[idx]
        resultados.append({
            "id": chunk["id"],
            "content": chunk["content"],
            "metadata": chunk["metadata"],
            "score_bm25": float(score),
        })

    return resultados


In [74]:
import math

def _normalizar_scores(valores):
    """
    Normaliza una lista de valores a [0, 1].
    Si todos son iguales, devuelve 1.0 para todos.
    """
    if not valores:
        return []

    vmin = min(valores)
    vmax = max(valores)

    if math.isclose(vmin, vmax):
        return [1.0 for _ in valores]

    return [(v - vmin) / (vmax - vmin) for v in valores]


### Búsqueda Híbrida: combinación BM25 + búsqueda semántica

La función `buscar_hibrida()` fusiona los resultados de BM25 y la búsqueda vectorial
semántica. Para cada chunk candidato se calculan dos puntajes:

- `score_bm25_norm`: puntaje normalizado de BM25
- `score_sem_norm`: puntaje normalizado de la similitud semántica

Luego se combinan mediante un hiperparámetro `alpha`.

Este mecanismo garantiza que:

- Si el usuario usa palabras exactas → BM25 aporta mayor peso.
- Si el usuario expresa una idea más conceptual → embeddings compensan mejor.

Finalmente se ordenan los chunks según el score híbrido y se devuelven los mejores candidatos.

In [57]:
def buscar_hibrida(
    consulta: str,
    k_bm25: int = 20,
    k_sem: int = 20,
    k_final: int = 10,
    alpha: float = 0.5,
    filtros: dict | None = None,
    devolver_embeddings: bool = False,
):
    """
    Búsqueda híbrida que combina:
    - BM25 (palabras clave)
    - Búsqueda vectorial (embeddings en Chroma)

    alpha controla el peso de BM25:
      score_hibrido = alpha * score_bm25_norm + (1 - alpha) * score_sem_norm
    """

    # 1) BM25
    res_bm25 = buscar_bm25(consulta, k=k_bm25)

    # 2) Búsqueda vectorial (ya existente)
    res_vec = buscar_vectorial(
        consulta,
        k=k_sem,
        filtros=filtros,
        devolver_embeddings=devolver_embeddings
    )

    # 3) Armar diccionario combinado por id de chunk
    combinado = {}

    # BM25
    for r in res_bm25:
        cid = r["id"]
        combinado.setdefault(cid, {
            "id": cid,
            "content": r["content"],
            "metadata": r["metadata"],
            "score_bm25": None,
            "score_sem": None,
            "score_hibrido": None,
        })
        combinado[cid]["score_bm25"] = r["score_bm25"]

    # Vectorial: usamos -distance como “score crudo” (menor distancia = mejor → mayor score)
    for r in res_vec:
        cid = r["id"]
        combinado.setdefault(cid, {
            "id": cid,
            "content": r["content"],
            "metadata": r["metadata"],
            "score_bm25": None,
            "score_sem": None,
            "score_hibrido": None,
        })
        # guardamos también por si querés usar distance directo
        combinado[cid]["distance"] = r["distance"]
        combinado[cid]["score_sem_raw"] = -float(r["distance"])
        if devolver_embeddings and "embedding_doc" in r:
            combinado[cid]["embedding_doc"] = r["embedding_doc"]

    # 4) Normalizar scores
    # BM25
    scores_bm25 = [
        v["score_bm25"]
        for v in combinado.values()
        if v["score_bm25"] is not None
    ]
    scores_sem_raw = [
        v["score_sem_raw"]
        for v in combinado.values()
        if "score_sem_raw" in v
    ]

    scores_bm25_norm = _normalizar_scores(scores_bm25)
    scores_sem_norm = _normalizar_scores(scores_sem_raw)

    # Mapear normalizados de vuelta a las entradas
    # (hacemos listas paralelas para simplificar)
    i_bm25 = 0
    for v in combinado.values():
        if v["score_bm25"] is not None:
            v["score_bm25_norm"] = scores_bm25_norm[i_bm25]
            i_bm25 += 1
        else:
            v["score_bm25_norm"] = 0.0

    i_sem = 0
    for v in combinado.values():
        if "score_sem_raw" in v:
            v["score_sem_norm"] = scores_sem_norm[i_sem]
            i_sem += 1
        else:
            v["score_sem_norm"] = 0.0

    # 5) Score híbrido
    for v in combinado.values():
        sb = v.get("score_bm25_norm", 0.0)
        ss = v.get("score_sem_norm", 0.0)
        v["score_hibrido"] = alpha * sb + (1 - alpha) * ss

    # 6) Ordenar y devolver top-k final
    lista = list(combinado.values())
    lista.sort(key=lambda x: x["score_hibrido"], reverse=True)

    return lista[:k_final]

## Re-Ranking con CrossEncoder

Luego de obtener los candidatos híbridos, se aplica una etapa de **re-ranking** utilizando
un modelo CrossEncoder (`ms-marco-MiniLM-L-2-v2`).  
A diferencia de los embeddings, el CrossEncoder analiza la consulta y el documento juntos,
permitiendo evaluar relaciones más profundas entre ambos.

El proceso consiste en:

1. Tomar los candidatos producidos por la búsqueda híbrida.
2. Evaluar cada par *(consulta, chunk)* con el CrossEncoder.
3. Ordenarlos nuevamente según este puntaje más preciso.

Este re-ranking mejora fuertemente la calidad de los primeros resultados (top-k),
siguiendo las recomendaciones estándar en sistemas RAG profesionales.

In [58]:
from txtai.pipeline import Similarity

# Modelo de CrossEncoder para rerank
crossencoder = Similarity(
    "cross-encoder/ms-marco-MiniLM-L-2-v2",
    crossencode=True,
    gpu=True
)


Device set to use cuda:0


In [59]:
def rerank_crossencoder(consulta: str, resultados: list[dict], top_k: int = 5):
    """
    Reordena los resultados usando un modelo cross-encoder.
    Toma:
      - consulta (texto)
      - resultados: lista de dicts con al menos 'content'
      - top_k: cuántos devolver

    Devuelve:
      - lista de dicts con campo extra 'score_rerank'
    """
    if not resultados:
        return []

    textos = [r["content"] for r in resultados]

    # Similarity devuelve lista de (id, score), ordenada por score desc
    pares = crossencoder(consulta, textos)

    rerankeados = []
    for idx, score in pares[:top_k]:
        item = resultados[idx].copy()
        item["score_rerank"] = float(score)
        rerankeados.append(item)

    return rerankeados


## Pipeline Vectorial Avanzado

Se define un pipeline completo para la fuente VECTORIAL:

1. **Búsqueda híbrida (BM25 + embeddings)**
2. **Re-ranking mediante CrossEncoder**
3. **Construcción de la respuesta final con un LLM**, usando como contexto solo los
   `k` chunks mejor posicionados.

Esta arquitectura representa un pipeline RAG consistente con sistemas de producción,
donde se combinan técnicas de recuperación lexical, recuperación semántica y
re-ranking profundo.

El resultado es un sistema robusto, capaz de recuperar información precisa incluso
cuando el usuario formula preguntas incompletas, vagas o parafraseadas.


In [75]:
import re


def extraer_producto_id_desde_query(query: str) -> str | None:
    """
    Busca un código de producto tipo P0001, P0199, etc. en el texto.
    Si no encuentra, devuelve None.
    """
    match = re.search(r"\bP\d{4}\b", query)
    if match:
        return match.group(0)
    return None


def pipeline_vectorial(
    query_usuario: str,
    k_bm25: int = 20,
    k_sem: int = 20,
    k_final: int = 5,
    alpha: float = 0.5,
    filtros: dict | None = None,
):
    """
    Pipeline completo para la fuente VECTORIAL:
    1) Recupera candidatos con búsqueda híbrida (BM25 + embeddings).
    2) (Opcional) Filtra por producto si se detecta un código P#### en la query.
    3) Re-rankea los candidatos con un CrossEncoder.
    4) Genera respuesta en lenguaje natural con el LLM.
    """

    # 0) Partimos de que NO hay filtros, salvo que el llamador nos haya pasado alguno
    filtros_final = filtros.copy() if filtros else None

    # 1) Intentar detectar un código de producto en la pregunta (P0001, P0199, etc.)
    producto_id = extraer_producto_id_desde_query(query_usuario)

    # Si detectamos producto_id, preparamos un filtro para la parte vectorial (Chroma)
    if producto_id is not None:
        filtros_producto = {
            "$or": [
                {"producto_id": producto_id},
                {"id_producto": producto_id},
            ]
        }

        if filtros_final:
            filtros_final = {"$and": [filtros_final, filtros_producto]}
        else:
            filtros_final = filtros_producto  # acá sí hay un filtro real

    # 2) Búsqueda híbrida (BM25 + vectorial)
    # Pedimos varios candidatos para que el rerank tenga de dónde elegir
    k_candidatos = max(k_final * 5, 25)

    candidatos = buscar_hibrida(
        consulta=query_usuario,
        k_bm25=k_bm25,
        k_sem=k_sem,
        k_final=k_candidatos,
        alpha=alpha,
        filtros=filtros_final,      # IMPORTANTE: puede ser dict o None, pero nunca {}
        devolver_embeddings=False,
    )

    # 3) Filtro post-hoc por producto_id en metadatos (para limpiar lo que vino de BM25)
    if producto_id is not None:
        candidatos_filtrados = [
            c for c in candidatos
            if c["metadata"].get("producto_id") == producto_id
            or c["metadata"].get("id_producto") == producto_id
        ]
        if candidatos_filtrados:
            candidatos = candidatos_filtrados

    # 4) Re-rank con CrossEncoder sobre todos los candidatos restantes
    rerankeados = rerank_crossencoder(
        consulta=query_usuario,
        resultados=candidatos,
        top_k=k_final,
    )

    # Si por algún motivo el rerank falla o da vacío, usamos los candidatos originales
    fragmentos_finales = rerankeados if rerankeados else candidatos[:k_final]

    # 5) Respuesta con LLM usando estos fragmentos
    respuesta = responder_desde_vectorial_con_llm(
        query_usuario,
        fragmentos_finales,
        client,
    )

    return {
        "fuente": "vectorial",
        "fragmentos": fragmentos_finales,
        "respuesta": respuesta,
    }


## Asistente integrado con búsqueda híbrida y ReRank

A partir de los componentes anteriores (BM25, búsqueda semántica, búsqueda híbrida
y re-ranqueo con CrossEncoder), se define una versión avanzada del asistente que
integra:

- Clasificador de intención (ML o LLM)
- Pipeline híbrido para la fuente vectorial
- Pipelines dinámicos para datos tabulares y grafo

In [76]:
def asistente_electro_avanzado(
    query_usuario: str,
    historial=None,
    k_vectorial: int = 5,
    metodo_clasificador: str = "ml"
):
    """
    Versión AVANZADA del asistente.

    Diferencias con asistente_electro (básico):
    - Para la fuente VECTORIAL usa el pipeline híbrido + rerank:
        pipeline_vectorial(...)
    - Para TABULAR y GRAFO reutiliza los mismos pipelines dinámicos.

    El clasificador de intención puede ser:
    - "ml"  -> clasificador entrenado TF-IDF + LogisticRegression
    - "llm" -> clasificador basado en Gemini few-shot
    """

    # 1) Clasificar la intención de la consulta
    fuente = clasificar_intencion(query_usuario, metodo=metodo_clasificador)

    # 2) Elegir pipeline según la fuente
    if fuente == "vectorial":
        # Ahora sí usamos la búsqueda híbrida + rerank + RAG
        resultado = pipeline_vectorial(
            query_usuario,
            k_bm25=20,
            k_sem=20,
            k_final=k_vectorial,
            alpha=0.5,
            filtros=None,
        )
        return resultado

    elif fuente == "tabular":
        return pipeline_tabular(query_usuario)

    elif fuente == "grafo":
        return pipeline_grafo(query_usuario)

    # Fallback por las dudas
    return {
        "fuente": "desconocida",
        "respuesta": (
            "No pude determinar a qué fuente de datos dirigir esta consulta. "
            "Probá reformular la pregunta indicando si buscás datos numéricos, "
            "instrucciones de uso o compatibilidad entre productos."
        )
    }


## Elección del LLM y Justificación del Entorno de Ejecución

Para este Trabajo Práctico se utiliza un modelo de lenguaje alojado **en la nube**, específicamente
el modelo **Gemini** de Google. La elección de un modelo en la nube se justifica por los
siguientes motivos:

1. **Limitaciones computacionales locales:**  
   Los modelos LLM modernos requieren GPU potentes y memoria significativa.
   Ejecutarlos localmente en un entorno como Google Colab o notebooks personales no es viable sin hardware especializado.

2. **Disponibilidad inmediata y sin configuración:**  
   El acceso a Gemini a través de API permite utilizar modelos avanzados sin necesidad de instalar pesos, configurar entornos complejos o gestionar almacenamiento.

3. **Mayor calidad y robustez:**  
   Los modelos en la nube suelen ser más grandes, más actualizados y con mejor
   rendimiento que los modelos locales livianos (ej: GPT2, LLaMA 7B, Mistral 7B).
   Esto es crucial para tareas como:
   - generación de código,
   - comprensión semántica,
   - clasificación de intención,
   - generación de texto en lenguaje natural.

4. **Escalabilidad y estabilidad:**  
   Los llamados al LLM se realizan de forma consistente y reproducible,
   asegurando que distintos usuarios obtengan resultados comparables.

---

### ¿Por qué no un modelo local?

Modelos locales como Ollama,no alcanzan el rendimiento necesario para:

- generar código correcto de Pandas o Cypher,  
- comprender consultas ambiguas,  
- reescribir respuestas con alta calidad.

Además, los modelos locales suelen carecer de:
- alineación fina para instrucciones,  
- robustez ante consultas ruidosas,  
- soporte multilingüe sólido.

Por estas razones, se opta por un modelo en la nube.

In [78]:
def llamar_llm(
    prompt: str,
    model: str = None,
    retries: int = 3
) -> str:
    """
    Llama al LLM de forma centralizada.

    Parámetros:
        prompt (str): Texto a enviar al LLM.
        model (str): Nombre del modelo a usar.
                     Si no se pasa, usa el modelo por defecto.
        retries (int): Cantidad de reintentos en caso de error.

    Devuelve:
        str: Respuesta del modelo o mensaje de error.
    """

    modelo = model if model is not None else modelo_gemini

    for i in range(retries):
        try:
            response = client.models.generate_content(
                model=modelo,
                contents=[prompt]
            )
            return response.text.strip()

        except Exception as e:
            print(f"⚠️ Error API (intento {i+1}/{retries}): {e}")
            time.sleep(2)

    return "⚠️ Error: El servicio de IA no está disponible en este momento."


## Justificación del Modelo de Lenguaje Utilizado (Gemini)

El modelo elegido para este trabajo es **Gemini**, por las siguientes razones:

1. **Excelente comprensión semántica en español:**  
   El proyecto requiere interpretar consultas naturales sobre productos, ventas, compatibilidad, etc. Gemini muestra un desempeño superior en comprensión en español comparado con modelos
   pequeños locales.

2. **Capacidad para generar código:**  
   En la fuente tabular y en la fuente grafo se necesita que el LLM genere:
   - filtros de Pandas,
   - consultas Cypher,
   - expresiones condicionales,
   - verificaciones básicas.

   Gemini demuestra alta precisión en generación de código ejecutable.

3. **Consistencia para clasificación de intención:**  
   El clasificador LLM basado en few-shot funciona con una precisión perfecta en las pruebas, lo cual confirma la robustez del modelo para tareas de categorización semántica.

4. **Velocidad y estabilidad:**  
   La API de Gemini permite un tiempo de respuesta adecuado y no requiere gestión de tokens compleja por parte del usuario.

5. **Integración con Python simplificada:**  
   La librería oficial `google-generativeai` facilita la creación de clientes y la interacción directa con el modelo.

---

### Conclusión

Gemini es una opción adecuada para este TP debido a:

- su rendimiento,
- su facilidad de integración,
- su soporte multilingüe,
- y su capacidad para generar código de calidad y respuestas contextuales.

Por estas razones, se adopta como LLM principal del sistema.


## Interacción conversacional interactiva

Para probar el sistema de manera más realista, se implementa un pequeño bucle
interactivo en el que el usuario puede escribir preguntas manualmente y finalizar
la sesión escribiendo `EXIT` o `SALIR`.

En cada turno:

1. El sistema clasifica la intención (vectorial / tabular / grafo).
2. Ejecuta el pipeline correspondiente (con búsqueda híbrida y re-ranking en el caso vectorial).
3. Genera una respuesta con el LLM.
4. Registra la pregunta, la respuesta y la fuente utilizada en el historial de la conversación.

Esto permite observar el comportamiento del asistente en un escenario similar al uso real
y verificar que la memoria y la integración de componentes funcionan correctamente.

In [79]:
def crear_asistente_conversacional(metodo_clasificador: str = "llm"):
    """
    Asistente conversacional oficial del TP.
    Usa SIEMPRE el asistente avanzado (búsqueda híbrida + rerank).
    Incluye historial para mantener memoria de la conversación.
    """

    historial = []

    def chat(pregunta: str):
        nonlocal historial

        resp = asistente_electro_avanzado(
            pregunta,
            historial=historial,
            metodo_clasificador=metodo_clasificador,
        )

        # Guardamos memoria
        historial.append({
            "pregunta": pregunta,
            "respuesta": resp["respuesta"],
            "fuente": resp["fuente"],
        })

        return resp, historial

    return chat


In [80]:
# Crear asistente conversacional
chat = crear_asistente_conversacional(metodo_clasificador="ml")

print("Asistente Electro (versión final)")
print("Escribí tu pregunta en español. Escribí EXIT o SALIR para terminar.\n")

while True:
    pregunta = input("Usuario: ").strip()

    if pregunta.lower() in ["exit", "salir", "quit"]:
        print("\nCerrando la conversación. Gracias por usar el asistente. 👋")
        break

    resp, historial = chat(pregunta)

    print(f"\nAsistente ({resp['fuente']}): {resp['respuesta']}\n")


Asistente Electro (versión final)
Escribí tu pregunta en español. Escribí EXIT o SALIR para terminar.

Usuario: ¿Cómo uso mi licuadora para hacer smoothies?

Asistente (vectorial): No tengo suficiente información para responder específicamente cómo usar tu licuadora para hacer smoothies. Las respuestas recuperadas mencionan que revises el manual del producto (código P0005 o P0006, dependiendo de si tu licuadora es HomeChef o ChefMaster) para más detalles sobre el uso correcto. Para obtener instrucciones precisas sobre cómo hacer smoothies, te sugiero que consultes el manual o contactes a nuestro servicio de atención al cliente.

Usuario: ¿Cuáles son las licuadoras de menos de $400?
🔧 Código generado por el modelo:

productos[(productos['nombre'].str.contains('Licuadora', case=False)) & (productos['precio_usd'] < 400)]

---


Asistente (tabular): Las siguientes licuadoras tienen un precio inferior a $400:

*   Licuadora TechHome (Blanca, 283.63 USD)
*   Plus Licuadora Pro TechHome (Negr